In [16]:
import glob
import sys
import os

from sklearn import svm
import numpy as np
from scipy.misc import imread

import hog

In [17]:
folder = 'INRIAPerson'
folder_suffix = '_64x128_H96'

In [18]:
def get_N_images(folder, folder_suffix, phase='train', validity='pos', N=100, verbose=False):
    path_regex = os.path.join(folder, phase + folder_suffix, validity, '*.png')
    array = []
    
    step = N // 4
    
    for n, img_path in enumerate(glob.iglob(path_regex)):
        if n > N:
            return array
        if verbose and n % step == 0:
            print('{}: {}/{}'.format(validity, n, N))
        array.append(get_image(img_path))
    
    return array


def get_image(img_path):
    return hog.get_hog_features(img_path)


def get_N_targets(validity='pos', N=100):
    if validity == 'pos':
        return np.ones((N, 1))
    return np.zeros((N, 1))

In [26]:
train_x = get_N_images(folder, folder_suffix, phase='train', validity='pos', verbose=True) +\
          get_N_images(folder, folder_suffix, phase='train', validity='neg', verbose=True)
train_x = np.array(train_x)
train_x = train_x.astype('float32')

train_y = np.concatenate((get_N_targets(validity='pos'), get_N_targets(validity='neg')))

pos: 0/100
pos: 25/100
pos: 50/100
pos: 75/100
pos: 100/100
neg: 0/100


/Users/arthur/code/scia/human-recognition/hog/hog.py:12: RuntimeWarning: invalid value encountered in true_divide
  vect = vect / norm


neg: 25/100
neg: 50/100
neg: 75/100
neg: 100/100


In [27]:
print(train_x.shape, train_y.shape)

(202, 3780) (200, 1)


In [28]:
np.isnan(train_x) == True

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ..., 
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], dtype=bool)

In [29]:
clf = svm.LinearSVC(C=0.01)
clf.fit(train_x, train_y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').